# Public Editor Convergence Tool 

In [1]:
import numpy as np
import pandas as pd
import re
from collections import Counter

## Goal: ## 
Our goal of this tool is to find the minimum number of users it takes to reach a consensus for each question. We are treating the consensus (the majority of answers) as the "correct answer". Then we are using a bootstrapping method in order to take simple random samples until we reach the correct answer of that specific question. 

## Reading In The Data and Organizing It ##
We'll need to read in our data file and the schema for it. As of right now, we are handling it file by file since each DataHuntAnswers.csv and Scheme.csv are different questions and answers. In the future, we hope we can build a tool to be able to read in all the data easily.

In [2]:
df_master = pd.read_csv('../newDataFormat/BETA_Language-2020-01-18T0225-DataHuntSubmitted.csv')
# Functions about schema is depreciated
# schema = pd.read_csv("../urap/Demo1Evi-2019-06-22T0023-Schema.csv")
# backup_schema = schema
df_master.head()

,schema_namespace,schema_sha256,quiz_task_uuid,task_url,tua_uuid,article_batch_name,article_number,article_filename,article_sha256,article_text_length,...,question_label,question_text,answer_label,answer_content,answer_uuid,submitted_tua_uuid,answer_text,case_number,highlight_count,submitted_tua
0,LanguageSpecialist_2019_08_08,383392eb525a877fee27c5e98a14f2ea089876892b30aa...,a25ed7d6-4f1b-40a4-9f72-54e13e5658eb,https://pe.goodlylabs.org/project/BETA_Languag...,9e7f8607-da75-45e1-b618-440fe3babe2d,Demo2,1712,1712SSSArticle12.txt,38428e24cdfa0392f3f5e1efda3253ffab6d7ed3051b1d...,5385,...,T1.Q1,Which of the following appears in the bolded t...,T1.Q1.A2,Exaggeration,c2b5506a-0062-4724-9ab9-1c7e46e0dda9,f599cea8-76fe-49f1-a8f1-4fa6408af22d,Exaggeration,1,2,"[{""case_number"": 1, ""text"": ""so shockingly hig..."
1,LanguageSpecialist_2019_08_08,383392eb525a877fee27c5e98a14f2ea089876892b30aa...,a25ed7d6-4f1b-40a4-9f72-54e13e5658eb,https://pe.goodlylabs.org/project/BETA_Languag...,9e7f8607-da75-45e1-b618-440fe3babe2d,Demo2,1712,1712SSSArticle12.txt,38428e24cdfa0392f3f5e1efda3253ffab6d7ed3051b1d...,5385,...,T1.Q1,Which of the following appears in the bolded t...,T1.Q1.A5,Shock or surprise,8334b531-5e7f-489b-85a9-4eb942eea67c,a80023e6-f062-4c79-b704-f17250c13fb0,Shock or surprise,1,1,"[{""case_number"": 1, ""text"": ""so shockingly hig..."
2,LanguageSpecialist_2019_08_08,383392eb525a877fee27c5e98a14f2ea089876892b30aa...,a25ed7d6-4f1b-40a4-9f72-54e13e5658eb,https://pe.goodlylabs.org/project/BETA_Languag...,9e7f8607-da75-45e1-b618-440fe3babe2d,Demo2,1712,1712SSSArticle12.txt,38428e24cdfa0392f3f5e1efda3253ffab6d7ed3051b1d...,5385,...,T1.Q1,Which of the following appears in the bolded t...,T1.Q1.A6,Slang,2f888961-c3fe-4e4e-937d-03fa3af5df6c,47710186-02b5-46ee-a1ca-b8463ace47ed,Slang,1,1,"[{""case_number"": 1, ""text"": ""wind up hooked"", ..."
3,LanguageSpecialist_2019_08_08,383392eb525a877fee27c5e98a14f2ea089876892b30aa...,a25ed7d6-4f1b-40a4-9f72-54e13e5658eb,https://pe.goodlylabs.org/project/BETA_Languag...,9e7f8607-da75-45e1-b618-440fe3babe2d,Demo2,1712,1712SSSArticle12.txt,38428e24cdfa0392f3f5e1efda3253ffab6d7ed3051b1d...,5385,...,T1.Q1,Which of the following appears in the bolded t...,T1.Q1.A12,Other problem with language,c8165d8a-9ef5-4ba2-a514-7e9273f9a960,cede2616-e9ae-4bcc-a0e2-91a953840b31,Other problem with language,1,1,"[{""case_number"": 1, ""text"": ""some minor proble..."
4,LanguageSpecialist_2019_08_08,383392eb525a877fee27c5e98a14f2ea089876892b30aa...,a25ed7d6-4f1b-40a4-9f72-54e13e5658eb,https://pe.goodlylabs.org/project/BETA_Languag...,9e7f8607-da75-45e1-b618-440fe3babe2d,Demo2,1712,1712SSSArticle12.txt,38428e24cdfa0392f3f5e1efda3253ffab6d7ed3051b1d...,5385,...,T1.Q3,How much does the exaggeration misrepresent re...,T1.Q3.A4,Minor misrepresentation,a2c927c0-f32d-478a-938c-d0f255fa5ff2,10c4569c-d4af-454f-9982-2f3ded5c575c,Minor misrepresentation,0,0,[]


We will be using the Schema file in order to help us disregard open-ended questions since those types of questions will not have a convergence or a majority answer.

In [3]:
# Depreciated, waiting to be fixed
# textQ = list(schema[schema['question_type'] == 'TEXT']["question_label"])
# textQ

The following code takes in the data and organizes it based on contributor ID. It shows which questions each user had reached as well as their answers to those respective questions.

In [4]:
'''
s: a series

Return:
    A dataframe, hot-encoded in a managable manner
'''
def hotcode_multiple_choices(s):
    keys = set()
    for elem in s:
        if type(elem) == list:
            for key in elem:
                keys.add(key)
    
    encode_data = np.zeros((len(s), len(keys)))
    i = 0
    for elem in s:
        if type(elem) == list:
            j = 0
            for key in keys:
                if key in elem:
                    encode_data[i][j] = 1
                j += 1
        i += 1
    encode_data = pd.DataFrame(encode_data, dtype=np.uint8)
    encode_data.columns = keys
    encode_data.index = s.index
    return encode_data

'''
This function get an article from 2020 feeds and convert it into the format of 2019 feeds so the code below can be reused

df: the dataframe of 2020 format, e.g. BETA_Language-2020-01-18T0225-DataHuntSubmitted.csv
article_id: article_number in df

Return:
    A pandas dataframe similiar to the format of 2019 feeds
'''
def getArticle(df, article_id):
    article = df[df['article_number'] == article_id]
    #We use quiz_taskrun_uuid as dummy index, since it is a primary index for 2019 feed
    tbl = pd.pivot_table(article, values='answer_label', index='quiz_taskrun_uuid', columns='question_label', aggfunc=list)
    for col in tbl.columns:
        helper = tbl[col].apply(lambda x: len(x) if type(x) == list else 0)
        if max(helper) <= 1: #just unwrap from list if everyone only choose one answer
            tbl[col] = tbl[col].apply(lambda x: x[0] if type(x) == list else 0)
        else:
            encoded = hotcode_multiple_choices(tbl[col])
            for col_ in encoded.columns:
                tbl[col_] = encoded[col_]
#             tbl = pd.concat([tbl, encoded], axis=0, join='inner', sort=False)
            tbl = tbl.drop(labels=col, axis=1)
    return tbl

## Bootstrapping ##
We used a Bootstrapping strategy in order to find the number it takes to match the consensus.

We chose a p-value of 0.01 where we want 99% of the time the majority reaches the "correct" consensus. 

In the future, we hope to integrate the user reputation tool and be able to put weights on the users. As of right now, we do not have the tool so the user weights/reputation scores are all just 1. 

The following function is to find the consensus of each question by finding the max of the answers. 

In [5]:
def getConsensus(answers):
    qcounts = Counter(answers)
    maxKey = max(qcounts.keys(), key=lambda key: qcounts[key])
    return maxKey

To help improve the effiency of our code, we converted all of the answers from strings to integers.

In [6]:
def strToInt(lst):
    vals = lst.unique()
    mapper = {vals[i]:i for i in range(len(vals))}
    return lst.replace(mapper)

The following function is to bootstrap and find the minimum number of users needed to find a convergence for each question. If we are unable to reach a confidence of 99% after sampling a max number of users, then we cannot find a convergence. The number of users that we sample can be changed in the future. 

In [8]:
# n = number of answer choices for questions
# c = consensus of entire dataset for question
# answers = answer column

def getN(questionName):
    answers = df[questionName].dropna()
    answers = strToInt(answers)
    n = len(pd.unique(answers))
    c = getConsensus(answers)
    
    for i in range(n + 1, max_group_size):
        count = 0
        for s in range(0, 1000):
            sample = np.random.choice(answers, i, replace=True)
            consensus = getConsensus(sample)
            if consensus == c:
                count += 1
        
        if count/1000 > .99:
            return i
    
#     return "Not converged"
    return -1 #Use -1 to represent not converged data, we are probably just ignoring it

We ran the function using a sample size of 100. We used a print statement in order to figure out the efficiency of it and organized the final results in a panda dataframe.

In [12]:
needed_group = 0

for article_id in df_master['article_number'].unique():
    df = getArticle(df_master, article_id)
    cols = ['Question', 'Min']
    lst=[]
    max_group_size = 100 # FEEL FREE TO CHANGE THIS

    for q in df.columns[1:]:
        if len(df[q].dropna().unique()) < 20:
            n = getN(q)
            lst.append(n)
            print('Article Number:', article_id, "Question:", q, "Converge:", n)
    max_needed = max(lst)
    if max_needed > needed_group:
        needed_group = max_needed

Article Number: 1712 Question: T1.Q11 Converge: 7
Article Number: 1712 Question: T1.Q12 Converge: 55
Article Number: 1712 Question: T1.Q13 Converge: 6
Article Number: 1712 Question: T1.Q14 Converge: -1
Article Number: 1712 Question: T1.Q15 Converge: -1
Article Number: 1712 Question: T1.Q2 Converge: 4
Article Number: 1712 Question: T1.Q3 Converge: -1
Article Number: 1712 Question: T1.Q5 Converge: 14
Article Number: 1712 Question: T1.Q7 Converge: 5
Article Number: 1712 Question: T1.Q9 Converge: -1
Article Number: 1712 Question: T1.Q1.A10 Converge: 7
Article Number: 1712 Question: T1.Q1.A12 Converge: 6
Article Number: 1712 Question: T1.Q1.A6 Converge: 29
Article Number: 1712 Question: T1.Q1.A4 Converge: 47
Article Number: 1712 Question: T1.Q1.A3 Converge: 5
Article Number: 1712 Question: T1.Q1.A5 Converge: 23
Article Number: 1712 Question: T1.Q1.A2 Converge: 49
Article Number: 1712 Question: T1.Q1.A9 Converge: 7
Article Number: 1712 Question: T1.Q1.A13 Converge: 3
Article Number: 1712 Que

In [13]:
print('We need {} people in this task'.format(needed_group))

We need 69 people in this task
